In [1]:
import pandas as pd
import numpy as np
import nltk 
from bs4 import BeautifulSoup            
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
import math
from textblob import TextBlob as tb
from sklearn.pipeline import Pipeline


In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head()


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'matches', 'background', 'colour', 'se...",<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestions', 'improvement',...",<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

20.742790699005127


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...","{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, matches, background, colour, seemingly, ...","{'aww': 1, 'matches': 1, 'background': 1, 'col..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...","{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestions, improvement, wondere...","{'make': 1, 'real': 1, 'suggestions': 1, 'impr..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]","{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
def getDataFrameFilterByToxicClass(df, toxicName):
    return train[train[toxicName] == 1]['listOfCleanWords']

toxicName = 'toxic'

clnTxtWrds = getDataFrameFilterByToxicClass(train, toxicName).tolist()

allTextsToxic = [' '.join(clnTxtWrds[x]) for x in range(len(clnTxtWrds))]


In [6]:
allTextsToxic[:10]

['cocksucker piss around work',
 'hey talk exclusive group wp talibans good destroying self appointed purist gang one asks questions abt anti social destructive non contribution wp ask sityush clean behavior issue nonsensical warnings',
 'bye look come think comming back tosser',
 'gay antisemmitian archangel white tiger meow greetingshhh uh two ways erased comment ww holocaust brutally slaying jews gays gypsys slavs anyone anti semitian shave head bald go skinhead meetings doubt words bible homosexuality deadly sin make pentagram tatoo forehead go satanistic masses gay pals first last warning fucking gay appreciate nazi shwain would write page wish talk anymore beware dark side',
 'fuck filthy mother ass dry',
 'sorry sorry screwed around someones talk page bad know templates talk page helps assert dominance know bow almighty administrators going go play outside mom',
 'get fucked get fuckeeed got drink cant put get fuck get fucked fucked right',
 'stupid peace shit stop deleting stuf

In [7]:
topNFeatures = 500
tdidfVectTrain = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrain = tdidfVectTrain.fit_transform(allTextsToxic)

In [8]:
selectedWordsTrain = tdidfVectTrain.get_feature_names()

In [9]:
# With all selected toxic words get BoW from all Texts 

clnTxtWrds = train['listOfCleanWords']
allTexts = [' '.join(clnTxtWrds[x]) for x in range(len(clnTxtWrds))]

vectTrain = CountVectorizer(vocabulary=selectedWordsTrain)
matrixBoWTrain = vectTrain.fit_transform(allTexts).toarray()

In [10]:
matrixBoWTrain.shape

(159571, 500)

In [11]:
matrixBoWTrain[6]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [12]:
np.where(matrixBoWTrain[6] == 1)

(array([ 27,  91, 338, 483], dtype=int64),)

In [13]:
selectedWordsTrain[:10]

['abusing',
 'account',
 'act',
 'actually',
 'add',
 'added',
 'admin',
 'administrator',
 'admins',
 'ah']

In [14]:
[selectedWordsTrain[x] for x in np.where(matrixBoWTrain[6] == 1)[0]]

['around', 'cocksucker', 'piss', 'work']

In [15]:
clnTxtWrds[6]

['cocksucker', 'piss', 'around', 'work']

In [16]:
print(matrixBoWTrain)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [17]:
test = pd.read_csv('../data/processed/' + nameTestCSV + '.csv', encoding='utf-8')
test.head()

,Unnamed: 0,id,comment_text,listOfCleanWords
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,"['yo', 'bitch', 'ja', 'rule', 'succesful', 'ev..."
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,"['rfc', 'title', 'fine', 'imo']"
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...","['sources', 'zawe', 'ashton', 'lapland']"
3,3,00017563c3f7919a,":If you have a look back at the source, the in...","['look', 'back', 'source', 'information', 'upd..."
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,"['anonymously', 'edit', 'articles']"


In [18]:
clnTxtWrdsTest = test['listOfCleanWords'].tolist()
allTextsTest = [' '.join(clnTxtWrdsTest[x]) for x in range(len(clnTxtWrdsTest))]

In [19]:
vectTest = CountVectorizer(vocabulary=selectedWordsTrain)
matrixBoWTest = vectTest.fit_transform(allTextsTest).toarray()

In [20]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(matrixBoWTrain, train['toxic'].tolist())

In [21]:
predicted = clf.predict(matrixBoWTest)

In [22]:
text_clf = Pipeline([('vect', CountVectorizer(vocabulary=selectedWordsTrain)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
                        ])
text_clf = text_clf.fit(allTexts, train['toxic'].tolist())

In [23]:
predicted2 = text_clf.predict(allTextsTest)


In [24]:
np.where(predicted == 1)

(array([], dtype=int64),)

In [25]:
np.where(predicted2 == 1)

(array([], dtype=int64),)

In [26]:
train['toxic']

0         0
1         0
2         0
3         0
4         0
5         0
6         1
7         0
8         0
9         0
10        0
11        0
12        1
13        0
14        0
15        0
16        1
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
159541    1
159542    0
159543    0
159544    0
159545    0
159546    1
159547    0
159548    0
159549    0
159550    0
159551    0
159552    0
159553    0
159554    1
159555    0
159556    0
159557    0
159558    0
159559    0
159560    0
159561    0
159562    0
159563    0
159564    0
159565    0
159566    0
159567    0
159568    0
159569    0
159570    0
Name: toxic, Length: 159571, dtype: int64

In [27]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 10) 
forest = forest.fit( matrixBoWTrain, train["toxic"].tolist())


In [28]:
result = forest.predict(matrixBoWTest)

In [29]:
result

array([0, 0, 0, ..., 0, 0, 0])

In [30]:
np.where(result == 1)

(array([], dtype=int64),)